In [2]:
# not using it anymore, see hrv_extraction.py

import neurokit2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('ggplot')
import datetime

df_time = pd.read_csv('foreground_background_differences.csv')
df_hrv = pd.read_csv(f'../dataset/P26/RRI.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'foreground_background_differences.csv'

In [ ]:
df_time

In [ ]:
df_hrv

In [ ]:
df_hrv.sort_values(by=['timestamp'])
df_hrv['timestamp_diff'] = -df_hrv.timestamp.diff(-1)
df_hrv.loc[df_hrv.index[-1], 'timestamp_diff'] = 0
df_hrv['date'] = pd.to_datetime(df_hrv.timestamp, unit='ms')

df_hrv

In [ ]:
loc = df_hrv.loc[(df_hrv.date > datetime.datetime(2019, 5, 12, 10, 0)) & (df_hrv.date <= datetime.datetime(2019, 5, 12, 11, 0))]

plt.figure(figsize=(25, 4))
plt.plot(loc.date, loc.interval)

In [ ]:
loc_time = df_time.loc[df_time.time_difference > 60 * 1000]

loc_time

In [ ]:
def get_key_timestamps(target, duration_ms, df, df_key='timestamp'):
    target_before = target - duration_ms
    target_after = target + duration_ms
    
    timestamp_before = 0
    timestamp_target = 0
    timestamp_after = 0
    
    # premise: df[df_key] is sorted
    for df_index, df_row in df.iterrows():
        if df_row[df_key] <= target_before:
            timestamp_before = df_row[df_key]
        elif df_row[df_key] <= target:
            timestamp_target = df_row[df_key]
        elif df_row[df_key] >= target_after:
            timestamp_after = df_row[df_key]
            break
    return timestamp_before, timestamp_target, timestamp_after
        
    

In [ ]:
df_analyse = pd.DataFrame()
df_analyse['RRI'] = df_hrv.interval
df_analyse['RRI_Time_ms'] = df_hrv.timestamp
df_analyse['RRI_Time'] = df_hrv.timestamp / 1000

df_analyse.sort_values(by='RRI_Time_ms')

dict_res = {
    'foreground_time': [],
    'before_RMSSD': [],
    'after_RMSSD': [],
    'RMSSD_diff': []
}

res = None

for index, row in loc_time.iterrows():
    foreground_time = row['foreground_time']
    background_time = row['background_time']

    (ts_before, ts_target, ts_after) = get_key_timestamps(foreground_time, 30 * 1000, df_analyse, df_key='RRI_Time_ms')

    if ts_before > 0 and ts_target > 0 and ts_after > 0:
        before_res = neurokit2.hrv_time(
            df_analyse.loc[(df_analyse['RRI_Time_ms'] >= ts_before) & (df_analyse['RRI_Time_ms'] <= ts_target)]
        )

        after_res = neurokit2.hrv_time(
            df_analyse.loc[(df_analyse['RRI_Time_ms'] > ts_target) & (df_analyse['RRI_Time_ms'] <= ts_after)]
        )

        res_row = pd.merge(before_res, after_res, suffixes=('_before', '_after'), left_index=True, right_index=True)
        
        if res is None: 
            res = res_row
        else:
            res = pd.concat([res, res_row], ignore_index=True)
            

In [ ]:
res